In [ ]:
def train_dqn(env, num_episodes=1000, batch_size=128, gamma=0.99, epsilon_start=1.0, epsilon_end=0.1, epsilon_decay=100):
    # 모델 초기화
    obs_shape = env.observation_space.shape[0] * env.observation_space.shape[1]
    n_actions = env.action_space.nvec[0] * env.action_space.nvec[1]
    policy_net = DQN(obs_shape, n_actions).to(device)
    target_net = DQN(obs_shape, n_actions).to(device)
    target_net.load_state_dict(policy_net.state_dict())
    target_net.eval()

    optimizer = optim.Adam(policy_net.parameters(), lr=1e-2)
    replay_buffer = ReplayBuffer(max_size=10000)

    epsilon = epsilon_start
    rewards_history = []
    steps_history = []

    success_count = 0  # 성공한 에피소드 카운트

    for episode in range(num_episodes):
        state = env.reset()
        state = torch.tensor(state.flatten(), dtype=torch.float, device=device)
        total_reward = 0
        steps = 0
        done = False

        while not done:
            # Epsilon-Greedy 행동 선택
            if random.random() < epsilon:
                action = [random.randint(0, env.action_space.nvec[0] - 1),
                          random.randint(0, env.action_space.nvec[1] - 1)]
            else:
                with torch.no_grad():
                    q_values = policy_net(state.unsqueeze(0))
                    action_idx = torch.argmax(q_values).item()
                    action = [action_idx // env.action_space.nvec[1], action_idx % env.action_space.nvec[1]]

            # 환경에서 한 스텝 실행
            next_state, reward, done, _ = env.step(action)
            next_state = torch.tensor(next_state.flatten(), dtype=torch.float, device=device)

            # 리플레이 메모리에 저장
            replay_buffer.push(state.cpu().numpy(), action[0] * env.action_space.nvec[1] + action[1], reward, next_state.cpu().numpy(), done)

            state = next_state
            total_reward += reward
            steps += 1

            # 학습
            if len(replay_buffer) >= batch_size:
                states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)

                # Q-Learning 대상 계산
                q_values = policy_net(states)
                q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)

                next_q_values = target_net(next_states).max(1)[0]
                targets = rewards + gamma * next_q_values * (1 - dones)

                # 손실 계산 및 역전파
                loss = nn.functional.mse_loss(q_values, targets)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        # 타겟 네트워크 업데이트
        if episode % 5 == 0:
            target_net.load_state_dict(policy_net.state_dict())

        # Epsilon 감소
        epsilon = max(epsilon_end, epsilon - (epsilon_start - epsilon_end) / epsilon_decay)

        # 에피소드 종료 후 성공 여부 확인
        if reward == 100:  # 성공한 경우
            success_count += 1

        rewards_history.append(total_reward)
        steps_history.append(steps)

        print(f"Episode {episode + 1}, Total Reward: {total_reward}, Steps: {steps}, Success Count: {success_count}, Epsilon: {epsilon:.2f}")

    return rewards_history, steps_history, success_count

# 환경 생성 및 학습 실행
env = RushHourEnv(vehicle_data)
rewards, steps, success_count = train_dqn(env)

# Train DQN Function

---

## 주요 구성 요소

### 1. **초기화**
- **정책 네트워크 (`policy_net`)**: 학습을 수행하는 메인 네트워크.
- **타겟 네트워크 (`target_net`)**: 고정된 Q-값을 제공하여 학습 안정성을 보장.
- **리플레이 버퍼 (`ReplayBuffer`)**: 환경과의 상호작용 데이터를 저장.

### 2. **Epsilon-Greedy 정책**
- 행동을 선택할 때 `epsilon` 확률로 랜덤 행동을 선택(탐험).
- 나머지 경우에는 Q-값이 가장 높은 행동을 선택(활용).

### 3. **학습 프로세스**
- 환경과 상호작용하며 `(state, action, reward, next_state, done)` 데이터를 수집.
- 리플레이 버퍼에서 샘플링한 데이터를 사용해:
  1. 정책 네트워크로 Q-값을 계산.
  2. 타겟 네트워크로 타겟 Q-값을 계산.
  3. 손실 함수(MSE)를 통해 네트워크 업데이트.

### 4. **타겟 네트워크 업데이트**
- 일정 간격으로(`episode % 5 == 0`) 정책 네트워크의 가중치를 타겟 네트워크에 복사.

### 5. **Epsilon 감소**
- 학습이 진행됨에 따라 탐험 비율을 점진적으로 줄여 활용 비율을 증가.

---

## 함수 정의

### **함수 시그니처**
```python
def train_dqn(
    env, 
    num_episodes=1000, 
    batch_size=128, 
    gamma=0.99, 
    epsilon_start=1.0, 
    epsilon_end=0.1, 
    epsilon_decay=100
)
